In [3]:
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/home/rahul/Downloads/dog vs cat/dataset/training_set"
test_dir="/home/rahul/Downloads/dog vs cat/dataset/test_set"


train_dogs= train_dir +"/dogs"
test_dogs=test_dir + "/dogs"
train_cats= train_dir + "/cats"
test_cats= test_dir +"/cats"


# To convert RGB images into array of numbers
data_generator = ImageDataGenerator(rescale = 1.0/255.0, zoom_range = 0.2)  


batch_size = 32
training_data = data_generator.flow_from_directory(directory = train_dir,
                                                   target_size = (64, 64),
                                                   batch_size = batch_size,
                                                   class_mode = 'binary')
testing_data = data_generator.flow_from_directory(directory = test_dir,
                                                  target_size = (64, 64),
                                                  batch_size = batch_size,
                                                  class_mode = 'binary')


from tensorflow.keras.applications import vgg16

vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape= training_data.image_shape)

from keras import layers, models, optimizers
model = models.Sequential()
model.add(vgg_conv)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

vgg_conv.trainable = False

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

history = model.fit(training_data,
                        steps_per_epoch = int(1000/batch_size),
                        epochs = 20,
                        validation_data = testing_data,
                        validation_steps = int(1000/batch_size)
                             )

from tensorflow.keras.preprocessing import image
import numpy as np
def testing_image(image_directory):
    test_image = image.load_img(image_directory, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(x = test_image)
    print(result)
    if result[0][0]  == 1:
        prediction = 'Dog'
    else:
        prediction = 'Cat'
    return prediction



Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
